In [1]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [2]:
!pip install torch -q
!pip install tqdm -q
!pip install -U bitsandbytes -q
!pip install transformers accelerate bitsandbytes -q
!pip install evaluate -q
!pip install datasets==2.21.0 -q # weird problem with load_metric
!pip install --upgrade huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 26.5 MB/s eta 0:00:00


In [3]:
from google.colab import files
import io
import pandas as pd
import json
from datasets import load_dataset
from evaluate import load
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForQuestionAnswering, AutoModelForCausalLM, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from tqdm import tqdm

# Custom dataset

In [4]:
uploaded = files.upload()
df = pd.read_csv('50_Q&A_Pairs.csv')
print(df.keys())
df.head(3)

df_filtered = df[['Question', 'Answer']]
df_filtered.rename(columns={"Question": "question", "Answer": "answer"}, inplace=True)
data_to_save = df_filtered.to_dict(orient='records')
# add id
data_to_save = [{**el, "id": str(idx)} for idx, el in enumerate(data_to_save)]
with open("50_Q&A_Pairs.json", 'w') as f:
  json.dump(data_to_save, f, indent=4)

custom_dataset = load_dataset("json", data_files="50_Q&A_Pairs.json", split="train")

Saving 50_Q&A_Pairs.csv to 50_Q&A_Pairs.csv
Index(['User Type', 'Question', 'Answer', 'Source [1]', '[2]', '[3]', '[4]'], dtype='object')


<ipython-input-4-dc83dc71448c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns={"Question": "question", "Answer": "answer"}, inplace=True)


Generating train split: 0 examples [00:00, ? examples/s]

# Gen datasets

In [5]:
# dataset = load_dataset("natural_questions", split="validation") # 140 GB disk space
dataset = load_dataset("nq_open", split="validation")
# dataset = load_dataset("squad_v2", split="validation") # https://paperswithcode.com/sota/question-answering-on-squad20
# dataset = load_dataset("ScienceQA", split="validation")

Generating train split:   0%|          | 0/87925 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3610 [00:00<?, ? examples/s]

In [6]:
dataset = dataset.map(lambda example, idx: {**example, "id": str(idx)}, with_indices=True)
dataset = dataset.map(lambda sample: {"answer": sample["answer"][0]})

Map:   0%|          | 0/3610 [00:00<?, ? examples/s]

Map:   0%|          | 0/3610 [00:00<?, ? examples/s]

# Models

In [8]:
model_name = "meta-llama/Llama-3.1-8B"
# model_name = "meta-llama/Llama-3.1-8B-Instruct"
# model_name = "universeTBD/astrollama"
# model_name = "Tijmen2/cosmosage-v3.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                              quantization_config=quantization_config,
                                              device_map='auto')

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [7]:
# model_repo = "eve-esa/eve-base-v0.1"
model_repo = "eve-esa/eve-instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_repo,
    device_map="auto",   # Uses available GPUs if any
    torch_dtype="auto",  # Automatically sets the correct tensor type
    use_safetensors=True, # Ensures it loads safetensors files
    quantization_config=quantization_config,
)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/885 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [8]:
# For reproducibility
torch.manual_seed(42)

# Generate and Evaluate

In [9]:
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id
)

In [10]:
# https://huggingface.co/spaces/evaluate-metric/squad_v2
metric = load("squad_v2")

In [11]:
def eval_qa(examples):
  questions = []
  preds = []
  refs = []

  for sample in tqdm(examples):
      question = sample["question"]
      generated_answer = generator(question, max_length=150, do_sample=True, truncation=True)[0]["generated_text"].replace(question, "").strip()

      questions.append(question)
      preds.append({"id": sample["id"], "prediction_text": generated_answer, "no_answer_probability": 0.})
      refs.append({"id": sample["id"], "answers": {"answer_start": [0], "text": sample["answer"]}})

  # Compute exact match and F1 scores
  final_score = metric.compute(predictions=preds, references=refs)

  return final_score

In [12]:
custom_score = eval_qa(custom_dataset) # 25-30 secs per sample
print("\n")
print(custom_score)

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 50/50 [18:01<00:00, 21.62s/it]




{'exact': 0.0, 'f1': 0.27869359210239975, 'total': 50, 'HasAns_exact': 0.0, 'HasAns_f1': 0.27869359210239975, 'HasAns_total': 50, 'best_exact': 0.0, 'best_exact_thresh': 0.0, 'best_f1': 0.27869359210239975, 'best_f1_thresh': 0.0}


In [13]:
score = eval_qa(dataset.select(list(range(100)))) # no need to eval on the entire set now, as it's sequential
print("\n")
print(score)

100%|██████████| 100/100 [17:10<00:00, 10.31s/it]



{'exact': 0.0, 'f1': 0.5363268758127453, 'total': 100, 'HasAns_exact': 0.0, 'HasAns_f1': 0.5363268758127453, 'HasAns_total': 100, 'best_exact': 0.0, 'best_exact_thresh': 0.0, 'best_f1': 0.5363268758127453, 'best_f1_thresh': 0.0}


# Scores

|           | Llama-3.1-8B | Llama-3.1-8B CPT | Llama-3.1-8B IFT ||
|----------|----------|----------|----------|----------|
| NQ_open   | 0./2.36     | 0./0.58     | 0./0.54     |
| EO QA testset   | 0./0.40     | 0./0.87     | 0./0.28     |